## Introduction

In this example, we train a model to learn to add two numbers, provided as strings.

**Example:**

- Input: "535+61"
- Output: "596"

Input may optionally be reversed, which was shown to increase performance in many tasks
 in: [Learning to Execute](http://arxiv.org/abs/1410.4615) and
[Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf).

Theoretically, sequence order inversion introduces shorter term dependencies between
 source and target for this problem.

**Results:**

For two digits (reversed):

+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits (reversed):

+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits (reversed):

+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits (reversed):

+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

14 digits (reversed):

+ Three layers LSTM (128 HN), 700k training examples = 99% train/test accuracy in 25 epochs



## Setup

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 800000
DIGITS = 21
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

## Generate the data

In [3]:

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

Generating data...
Total questions: 800000


## Vectorize the data

In [4]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Vectorization...


<ipython-input-4-195dd5dc8a21>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
<ipython-input-4-195dd5dc8a21>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)


Training Data:
(720000, 43, 12)
(720000, 22, 12)
Validation Data:
(80000, 43, 12)
(80000, 22, 12)


## Build the model

In [5]:
print("Build model...")
num_layers = 3  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 22, 128)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 22, 128)           131584    
                                                                 
 lstm_2 (LSTM)               (None, 22, 128)           131584    
                                                                 
 lstm_3 (LSTM)               (None, 22, 128)           131584    
                                                                 
 dense (Dense)               (None, 22, 12)            1548      
                                         

## Train the model

In [6]:
from tqdm.auto import trange
from tqdm.keras import TqdmCallback

epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in trange(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        callbacks=[TqdmCallback(verbose=1)],
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)

  0%|          | 0/29 [00:00<?, ?it/s]


Iteration 1


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

1/1 [==============================] - 1s 1s/step
Q 307166+7856                                 T 315022                 ☒ 315885                
1/1 [==============================] - 0s 20ms/step
Q 9960806+44407                               T 10005213               ☒ 90002282              
1/1 [==============================] - 0s 19ms/step
Q 667636774427924053521+35754701000236190036  T 703391475428160243557  ☒ 675888888888888888885 
1/1 [==============================] - 0s 23ms/step
Q 429211377036634831511+847301                T 429211377036635678812  ☒ 42921137888888888888  
1/1 [==============================] - 0s 22ms/step
Q 2720+46411251957                            T 46411254677            ☒ 46411258285           
1/1 [==============================] - 0s 23ms/step
Q 27793+9565000950529936                      T 9565000950557729       ☒ 9565000988888882      
1/1 [==============================] - 0s 21ms/step
Q 659125129+4016384327776                     T 4017043452905 

...

0batch [00:00, ?batch/s]

1/1 [==============================] - 0s 27ms/step
Q 786755091588933764+62                       T 786755091588933826     ☑ 786755091588933826    
1/1 [==============================] - 0s 29ms/step
Q 435493246+9996007892                        T 10431501138            ☒ 10411383336           
1/1 [==============================] - 0s 31ms/step
Q 819475+490                                  T 819965                 ☑ 819965                
1/1 [==============================] - 0s 31ms/step
Q 4523534+248741106878494629                  T 248741106883018163     ☒ 248741106883019888    
1/1 [==============================] - 0s 30ms/step
Q 6+613329491699002                           T 613329491699008        ☑ 613329491699008       
1/1 [==============================] - 0s 30ms/step
Q 28653617980+425608616                       T 29079226596            ☒ 29079383338           
1/1 [==============================] - 0s 32ms/step
Q 73577613514116899+340403052998              T 735779539171

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

1/1 [==============================] - 0s 21ms/step
Q 55168072+690013701032                       T 690068869104           ☒ 690068999999          
1/1 [==============================] - 0s 34ms/step
Q 90918675780615+71                           T 90918675780686         ☑ 90918675780686        
1/1 [==============================] - 0s 20ms/step
Q 6462611280697858300+129272                  T 6462611280697987572    ☒ 6462611280697987699   
1/1 [==============================] - 0s 26ms/step
Q 166973066552526642+263930846854             T 166973330483373496     ☒ 166973330499999999    
1/1 [==============================] - 0s 33ms/step
Q 6+1293687580060980376                       T 1293687580060980382    ☒ 1293687580060980389   
1/1 [==============================] - 0s 21ms/step
Q 68631+1672                                  T 70303                  ☑ 70303                 
1/1 [==============================] - 0s 27ms/step
Q 73696013124946+70100664694978821027         T 701007383909

You'll get to 99+% validation accuracy after ~30 epochs.

Example available on HuggingFace.

| Trained Model | Demo |
| :--: | :--: |
| [![Generic badge](https://img.shields.io/badge/🤗%20Model-Addition%20LSTM-black.svg)](https://huggingface.co/keras-io/addition-lstm) | [![Generic badge](https://img.shields.io/badge/🤗%20Spaces-Addition%20LSTM-black.svg)](https://huggingface.co/spaces/keras-io/addition-lstm) |

In [ ]:
model.save("model_21_800k.keras")